Iterative Solvers
===

So far we have used direct solvers to solve the linear system of equations. Although a direct solver can profit from the sparse matrix, it's arithmetic complexity is sub-optimal. For large-scale problems iterative solvers are a must.

The conjugate gradient (cg) method is the standard method for symmetric and positive definite matrices. It's convergence rate depends on a preconditioner, what is a cheap approximative inverse to the matrix.

In [1]:
import netgen.gui
from ngsolve import *
%gui tk

We generate a 3D geometry and mesh using the constructive solid geometry (CSG) modeler of Netgen:

In [10]:
from netgen.csg import *
geo = CSGeometry()
cube = OrthoBrick (Pnt(0,0,0), Pnt(1,1,1)).bc("outer")
cyl = Cylinder( Pnt(1,0.5,0.5), Pnt(0,0.5,0.5), 0.2).bc("inner")
geo.Add (cube-cyl, col=(0,0,1))
geo.Draw()

mesh = Mesh(geo.GenerateMesh(maxh=0.1))
mesh.Refine()
mesh.Refine()
mesh.Curve(3)
Draw (mesh)

In [11]:
fes = H1(mesh, order=3, dirichlet="outer", flags = { "wb_withoutedges" : True })
print ("we have", fes.ndof, "unknowns")
u = fes.TrialFunction()
v = fes.TestFunction()

a = BilinearForm(fes)
a += SymbolicBFI(grad(u)*grad(v))

f = LinearForm(fes)
f += SymbolicLFI(v)

# c = Preconditioner(a, "local")
# c = Preconditioner(a, "direct", inverse="sparsecholesky")
c = Preconditioner(a, "bddc")

gfu = GridFunction(fes)

we have 1589538 unknowns


assemble system and setup preconditioner in parallel:

In [12]:
ngsglobals.msg_level=5
with TaskManager():
    a.Assemble()
    f.Assemble()

solve the system using the preconditioned conjugate gradient method:

In [13]:
from ngsolve.solvers import CG

with TaskManager():
    CG (mat=a.mat, pre=c.mat, rhs=f.vec, sol=gfu.vec, 
        printrates=True, maxsteps=200)
Draw(gfu)

it =  0  err =  0.28862781630691464
it =  1  err =  0.10684320380282808
it =  2  err =  0.08694568937068343
it =  3  err =  0.07161777381046253
it =  4  err =  0.05866794979024328
it =  5  err =  0.045272293052670845
it =  6  err =  0.028589895087126477
it =  7  err =  0.018831507044259222
it =  8  err =  0.014428572396744301
it =  9  err =  0.009424883508505082
it =  10  err =  0.006399809095338783
it =  11  err =  0.004819552146547561
it =  12  err =  0.0033374515130286594
it =  13  err =  0.002240916551297694
it =  14  err =  0.0015146186589075226
it =  15  err =  0.0010769777263741885
it =  16  err =  0.0007714379534315349
it =  17  err =  0.0005107855477309455
it =  18  err =  0.00034380924632675336
it =  19  err =  0.00023760166679236342
it =  20  err =  0.00015933394463376794
it =  21  err =  0.000104679585109688
it =  22  err =  6.986881902040638e-05
it =  23  err =  4.73089653954801e-05
it =  24  err =  3.192110983386464e-05
it =  25  err =  2.1075175839086192e-05
it =  26  er